In [2]:
from neuroglancer_chat.backend.tools import neuroglancer_state

# S3

In [ ]:
import json
import urllib.parse
import re
from typing import Callable, Tuple, Any, Mapping, Optional

try:
    import boto3  # Optional; only needed for direct s3:// fetch
    _HAS_BOTO3 = True
except ImportError:
    _HAS_BOTO3 = False

# -------- Core Helpers --------

def _is_probably_json(text: str) -> bool:
    text = text.strip()
    return text.startswith('{') and text.endswith('}')

def _percent_decode(fragment: str) -> str:
    # Neuroglancer usually just uses encodeURIComponent, so normal unquote
    return urllib.parse.unquote(fragment)

def _percent_encode_minified(obj: Mapping[str, Any]) -> str:
    # Neuroglancer state links usually store minified JSON
    raw = json.dumps(obj, separators=(',', ':'), sort_keys=False)
    # Safe chars: keep a small set unescaped (tolerant). Adjust if you want stricter.
    return urllib.parse.quote(raw, safe="!~*'()")

def _fetch_http(url: str, http_get: Optional[Callable[[str], str]] = None) -> str:
    if http_get:
        return http_get(url)
    # Default simple implementation
    import urllib.request
    with urllib.request.urlopen(url) as resp:
        return resp.read().decode('utf-8')

def _fetch_s3(url: str) -> str:
    """
    Fetch s3://bucket/key using boto3 (if available).
    """
    if not _HAS_BOTO3:
        raise RuntimeError("boto3 not installed; install boto3 or provide a custom fetcher.")
    m = re.match(r'^s3://([^/]+)/(.+)$', url)
    if not m:
        raise ValueError(f"Not a valid s3 URL: {url}")
    bucket, key = m.group(1), m.group(2)
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    return obj['Body'].read().decode('utf-8')

def _default_fetch(url: str) -> str:
    if url.startswith('s3://'):
        return _fetch_s3(url)
    elif url.startswith('http://') or url.startswith('https://'):
        return _fetch_http(url)
    else:
        raise ValueError(f"Unsupported pointer scheme for URL fetch: {url}")

# -------- Public API --------

def resolve_neuroglancer_pointer(
    fragment: str,
    fetcher: Callable[[str], str] | None = None
) -> Tuple[dict, bool]:
    """
    Resolve a Neuroglancer fragment (the part after '#!') into a state dict.

    Parameters
    ----------
    fragment : str
        Either inline (percent-encoded) JSON or a pointer URL (http(s)://, s3://).
    fetcher : callable, optional
        Custom fetcher(url) -> text. If not provided, uses default logic.

    Returns
    -------
    state_dict : dict
    was_pointer : bool
        True if we had to fetch it from a pointer; False if inline JSON.
    """
    # Raw fragment may still be percent-encoded
    decoded = _percent_decode(fragment)

    # Case 1: Inline JSON
    if _is_probably_json(decoded):
        try:
            return json.loads(decoded), False
        except json.JSONDecodeError as e:
            raise ValueError(f"Fragment looked like JSON but failed to parse: {e}") from e

    # Case 2: Pointer (URL to JSON)
    fetcher = fetcher or _default_fetch
    json_text = fetcher(decoded)
    try:
        state = json.loads(json_text)
    except json.JSONDecodeError as e:
        raise ValueError(f"Fetched text is not valid JSON from pointer '{decoded}': {e}") from e
    return state, True

def neuroglancer_state_to_url(
    state: Mapping[str, Any],
    viewer_base_url: str
) -> str:
    """
    Serialize a Neuroglancer state dict to a canonical inline URL.
    """
    state.pop('ng_link', None)  # Remove pointer if present
    frag = _percent_encode_minified(state)
    if viewer_base_url.endswith('/'):
        viewer_base_url = viewer_base_url.rstrip('/')
    return f"{viewer_base_url}/#!{frag}"

def expand_if_pointer_and_generate_inline(
    full_url: str,
    fetcher: Callable[[str], str] | None = None
) -> Tuple[str, dict, bool]:
    """
    Given a full viewer URL (or just a fragment), resolve and produce
    the canonical inline state URL.

    Returns
    -------
    canonical_url : str
    state_dict : dict
    was_pointer : bool
    """
    # Split base and fragment
    if '#!' in full_url:
        base, fragment = full_url.split('#!', 1)
    else:
        # If only fragment provided
        base, fragment = '', full_url.lstrip('#!')
    state, was_pointer = resolve_neuroglancer_pointer(fragment, fetcher=fetcher)
    canonical = neuroglancer_state_to_url(state, base or "https://your-neuroglancer.example")
    return canonical, state, was_pointer

In [6]:
pointer_fragment = "s3://aind-open-data/HCR_783884-02_2025-08-21_13-00-00/raw_data.json"
viewer = "https://aind-neuroglancer-sauujisjxq-uw.a.run.app"
canonical_url, state, was_pointer = expand_if_pointer_and_generate_inline(f"{viewer}/#!{pointer_fragment}")
print(canonical_url)
print(state)
print(was_pointer)

https://aind-neuroglancer-sauujisjxq-uw.a.run.app/#!%7B%22dimensions%22%3A%7B%22x%22%3A%5B2.466335423895654e-07%2C%22m%22%5D%2C%22y%22%3A%5B2.466335423895654e-07%2C%22m%22%5D%2C%22z%22%3A%5B1e-06%2C%22m%22%5D%2C%22c'%22%3A%5B1%2C%22%22%5D%2C%22t%22%3A%5B0.001%2C%22s%22%5D%7D%2C%22layers%22%3A%5B%7B%22type%22%3A%22image%22%2C%22source%22%3A%5B%7B%22url%22%3A%22zarr%3A%2F%2Fs3%3A%2F%2Faind-open-data%2FHCR_783884-02_2025-08-21_13-00-00%2FSPIM.ome.zarr%2FTile_X_0000_Y_0000_Z_0000_ch_405.zarr%22%2C%22transform%22%3A%7B%22matrix%22%3A%5B%5B1.0%2C0.0%2C0.0%2C0.0%2C0.0%2C0.0%5D%2C%5B0.0%2C1.0%2C0.0%2C0.0%2C0.0%2C0.0%5D%2C%5B0.0%2C0.0%2C1.0%2C0.0%2C0.0%2C0.0%5D%2C%5B0.0%2C0.0%2C0.0%2C1.0%2C0.0%2C-4560.0%5D%2C%5B0.0%2C0.0%2C0.0%2C0.0%2C1.0%2C-1824.0%5D%5D%2C%22outputDimensions%22%3A%7B%22t%22%3A%5B0.001%2C%22s%22%5D%2C%22c'%22%3A%5B1%2C%22%22%5D%2C%22z%22%3A%5B1e-06%2C%22m%22%5D%2C%22y%22%3A%5B2.466335423895654e-07%2C%22m%22%5D%2C%22x%22%3A%5B2.466335423895654e-07%2C%22m%22%5D%7D%7D%7D%2C%7B%22u

In [3]:
import boto3, botocore
session = boto3.Session()  # or boto3.Session(profile_name="my-sso")
sts = session.client("sts")
print(sts.get_caller_identity())
print("Profile name:", session.profile_name)
print("Region:", session.region_name)
print("Access key (partial):", session.get_credentials().access_key[:4], "...")

NoCredentialsError: Unable to locate credentials

In [3]:
nglink = "https://aind-neuroglancer-sauujisjxq-uw.a.run.app/#!%7B%22dimensions%22:%7B%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22t%22:%5B0.001%2C%22s%22%5D%7D%2C%22position%22:%5B48.5%2C-5423.5%2C584.5%2C0.5%5D%2C%22crossSectionScale%22:1%2C%22projectionScale%22:8192%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%5B%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0000_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-6384%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0001_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-4559%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0002_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-2735%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0003_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-912%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0004_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C912%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0005_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C2735%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0006_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C4559%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0007_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C6384%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0000_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-6384%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0001_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-4559%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0002_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-2735%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0003_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-912%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0004_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C912%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0005_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C2735%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0006_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C4559%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0007_Z_0000_ch_405.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C6384%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%5D%2C%22localDimensions%22:%7B%22c%27%22:%5B1%2C%22%22%5D%7D%2C%22localPosition%22:%5B0.5%5D%2C%22tab%22:%22source%22%2C%22opacity%22:1%2C%22blend%22:%22additive%22%2C%22shader%22:%22#uicontrol%20vec3%20color%20color%28default=%5C%22#ffffff%5C%22%29%5Cn#uicontrol%20invlerp%20normalized%5Cnvoid%20main%28%29%20%7B%5CnemitRGB%28color%20%2A%20normalized%28%29%29%3B%5Cn%7D%22%2C%22shaderControls%22:%7B%22normalized%22:%7B%22range%22:%5B90%2C1200%5D%7D%7D%2C%22name%22:%22CH_405%22%7D%2C%7B%22type%22:%22image%22%2C%22source%22:%5B%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0000_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-6384%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0001_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-4559%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0002_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-2735%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0003_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-912%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0004_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C912%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0005_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C2735%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0006_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C4559%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0007_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C6384%5D%2C%5B0%2C0%2C0%2C0%2C1%2C-912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0000_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-6384%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0001_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-4559%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0002_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-2735%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0003_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C-912%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0004_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C912%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0005_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C2735%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0006_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C4559%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%2C%7B%22url%22:%22zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0001_Y_0007_Z_0000_ch_638.zarr%22%2C%22transform%22:%7B%22matrix%22:%5B%5B1%2C0%2C0%2C0%2C0%2C0%5D%2C%5B0%2C1%2C0%2C0%2C0%2C0%5D%2C%5B0%2C0%2C1%2C0%2C0%2C0%5D%2C%5B0%2C0%2C0%2C1%2C0%2C6384%5D%2C%5B0%2C0%2C0%2C0%2C1%2C912%5D%5D%2C%22outputDimensions%22:%7B%22t%22:%5B0.001%2C%22s%22%5D%2C%22c%27%22:%5B1%2C%22%22%5D%2C%22z%22:%5B0.000001%2C%22m%22%5D%2C%22y%22:%5B2.4505557964997545e-7%2C%22m%22%5D%2C%22x%22:%5B2.4505557964997545e-7%2C%22m%22%5D%7D%7D%7D%5D%2C%22localDimensions%22:%7B%22c%27%22:%5B1%2C%22%22%5D%7D%2C%22tab%22:%22source%22%2C%22opacity%22:1%2C%22blend%22:%22additive%22%2C%22shader%22:%22#uicontrol%20vec3%20color%20color%28default=%5C%22#ff00ff%5C%22%29%5Cn#uicontrol%20invlerp%20normalized%5Cnvoid%20main%28%29%20%7B%5CnemitRGB%28color%20%2A%20normalized%28%29%29%3B%5Cn%7D%22%2C%22shaderControls%22:%7B%22normalized%22:%7B%22range%22:%5B90%2C1200%5D%7D%7D%2C%22name%22:%22CH_638%22%7D%5D%2C%22showAxisLines%22:false%2C%22showScaleBar%22:false%2C%22layout%22:%224panel%22%7D"
neuroglancer_state.from_url(nglink)

{'dimensions': {'x': [2.4505557964997545e-07, 'm'],
  'y': [2.4505557964997545e-07, 'm'],
  'z': [1e-06, 'm'],
  't': [0.001, 's']},
 'position': [48.5, -5423.5, 584.5, 0.5],
 'crossSectionScale': 1,
 'projectionScale': 8192,
 'layers': [{'type': 'image',
   'source': [{'url': 'zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0000_Z_0000_ch_405.zarr',
     'transform': {'matrix': [[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, -6384],
       [0, 0, 0, 0, 1, -912]],
      'outputDimensions': {'t': [0.001, 's'],
       "c'": [1, ''],
       'z': [1e-06, 'm'],
       'y': [2.4505557964997545e-07, 'm'],
       'x': [2.4505557964997545e-07, 'm']}}},
    {'url': 'zarr://s3://aind-open-data/HCR_794300-10_2025-08-14_13-00-00/SPIM.ome.zarr/Tile_X_0000_Y_0001_Z_0000_ch_405.zarr',
     'transform': {'matrix': [[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, -4